# Feed Forward Neural Network

In [4]:
# imports
import torch
import warnings
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.datasets as dsets
import torchvision.transforms as transforms

warnings.filterwarnings('ignore', category=FutureWarning)

print(f'torch version is : {torch.__version__}')
print(f'numpy version is : {np.__version__}')

torch version is : 2.0.0+cpu
numpy version is : 1.23.5


In [5]:
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor(),
                           download=True)

100%|██████████| 9912422/9912422 [00:25<00:00, 384935.75it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 802738.80it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|██████████| 1648877/1648877 [00:03<00:00, 442859.27it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 4542/4542 [00:00<00:00, 4545580.71it/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [6]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [7]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 

        # Non-linearity
        self.sigmoid = nn.Sigmoid()

        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        # Linear function  # LINEAR
        out = self.fc1(x)

        # Non-linearity  # NON-LINEAR
        out = self.sigmoid(out)

        # Linear function (readout)  # LINEAR
        out = self.fc2(out)
        return out

In [8]:
input_dim = 28 * 28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

In [9]:
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

## Print Model Attributes

In [12]:
print(model.parameters)
print(model.parameters())

<bound method Module.parameters of FeedforwardNeuralNetModel(
  (fc1): Linear(in_features=784, out_features=100, bias=True)
  (sigmoid): Sigmoid()
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)>
<generator object Module.parameters at 0x00000271E56EBAC0>


In [13]:
print(len(list(model.parameters())))

4


In [14]:
# FC 1 Parameters 
print(list(model.parameters())[0].size())

torch.Size([100, 784])


In [15]:
# FC 1 Bias Parameters
print(list(model.parameters())[1].size())

torch.Size([100])


In [16]:
# FC 2 Parameters
print(list(model.parameters())[2].size())

torch.Size([10, 100])


## Train Model

In [18]:
criterion = nn.CrossEntropyLoss()

In [19]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images with gradient accumulation capabilities
        images = images.view(-1, 28*28).requires_grad_()
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        # Forward pass to get output/logits
        outputs = model(images)
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        # Getting gradients w.r.t. parameters
        loss.backward()
        # Updating parameters
        optimizer.step()
        iter += 1
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images with gradient accumulation capabilities
                images = images.view(-1, 28*28).requires_grad_()
                # Forward pass only to get logits/output
                outputs = model(images)
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                # Total number of labels
                total += labels.size(0)
                # Total correct predictions
                correct += (predicted == labels).sum()
            accuracy = 100 * correct / total
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 0.6333867907524109. Accuracy: 85.72000122070312
Iteration: 1000. Loss: 0.3981088697910309. Accuracy: 89.61000061035156
Iteration: 1500. Loss: 0.3107844293117523. Accuracy: 90.4000015258789
Iteration: 2000. Loss: 0.4128163158893585. Accuracy: 91.22000122070312
Iteration: 2500. Loss: 0.2589390277862549. Accuracy: 91.66999816894531
Iteration: 3000. Loss: 0.4749063551425934. Accuracy: 92.12999725341797
